# Exploring Magic The Gathering.

In [ ]:
import pandas as pd
import duckdb as db
import numpy as np

In [ ]:
df = pd.read_json('oracle-cards-20250320090436.json') #03/20/2025 https://scryfall.com/docs/api/bulk-data Oracle Cards File

In [ ]:
df.columns.to_list()

['object',
 'id',
 'oracle_id',
 'multiverse_ids',
 'mtgo_id',
 'tcgplayer_id',
 'cardmarket_id',
 'name',
 'lang',
 'released_at',
 'uri',
 'scryfall_uri',
 'layout',
 'highres_image',
 'image_status',
 'image_uris',
 'mana_cost',
 'cmc',
 'type_line',
 'oracle_text',
 'power',
 'toughness',
 'colors',
 'color_identity',
 'keywords',
 'all_parts',
 'legalities',
 'games',
 'reserved',
 'game_changer',
 'foil',
 'nonfoil',
 'finishes',
 'oversized',
 'promo',
 'reprint',
 'variation',
 'set_id',
 'set',
 'set_name',
 'set_type',
 'set_uri',
 'set_search_uri',
 'scryfall_set_uri',
 'rulings_uri',
 'prints_search_uri',
 'collector_number',
 'digital',
 'rarity',
 'watermark',
 'flavor_text',
 'card_back_id',
 'artist',
 'artist_ids',
 'illustration_id',
 'border_color',
 'frame',
 'frame_effects',
 'security_stamp',
 'full_art',
 'textless',
 'booster',
 'story_spotlight',
 'edhrec_rank',
 'preview',
 'prices',
 'related_uris',
 'purchase_uris',
 'mtgo_foil_id',
 'penny_rank',
 'arena_id

In [ ]:
#Removes '*', '?', '∞' and X values from power and toughness
def clean_numeric_values(df):
    def to_numeric(value):
        if str(value).isnumeric():
            return int(value)
        else:
            return np.nan
    df['power'] = df['power'].apply(to_numeric)
    df['toughness'] = df['toughness'].apply(to_numeric)

    return df

df = clean_numeric_values(df)

In [ ]:
df['set_name'] = df['set_name'].str.upper()


In [477]:
df['mana_cost'] = df['mana_cost'].apply(lambda x: "{}" if x == "" else x)

## 1. Questions I want to answer:
1. Wich mana color or mana combination has more cards? Ans: Hard to know it

2. Wich mana color or mana combination has the card with bigger power and toughness? Ans: Probably green though

3. What is the distribution of rarities (rarity) across different sets (set_name)?

4. What are the most common keywords (keywords) used across all cards?

5. Which cards have the highest number of keyword abilities (keywords)?

6. What are the longest and shortest card names (name)?

7. Which card has the longest flavour text?

In [ ]:
#First Question
my_query =  '''
    SELECT 
        color_identity,
        COUNT(oracle_id) AS amount 
    FROM df 
    GROUP BY color_identity
    ORDER BY amount DESC;
    '''

results = db.query(my_query).to_df()
print(results)


     color_identity  amount
0                []    5589
1               [B]    4710
2               [W]    4710
3               [R]    4650
4               [U]    4608
5               [G]    4579
6            [B, U]     480
7            [U, W]     478
8            [B, R]     478
9            [G, W]     461
10           [G, R]     450
11           [R, W]     429
12           [R, U]     412
13           [G, U]     409
14           [B, W]     407
15           [B, G]     405
16  [B, G, R, U, W]     140
17        [G, R, W]     116
18        [B, R, U]     112
19        [B, U, W]      99
20        [B, G, R]      95
21        [G, U, W]      95
22        [R, U, W]      73
23        [B, G, U]      69
24        [B, R, W]      67
25        [G, R, U]      63
26        [B, G, W]      54
27     [G, R, U, W]       8
28     [B, G, U, W]       4
29     [B, G, R, U]       3
30     [B, G, R, W]       2
31     [B, R, U, W]       2


In [ ]:
#Second Question
my_query = ''' 
  SELECT 
    name,
    color_identity,
    power, 
    toughness
FROM df
WHERE
  power != 'Nan'
  AND toughness != 'Nan'
ORDER BY power+toughness DESC
LIMIT(20);
'''
results = db.query(my_query).to_df()
print(results)

                             name color_identity  power  toughness
0      B.F.M. (Big Furry Monster)            [B]   99.0       99.0
1                      Marit Lage            [B]   20.0       20.0
2            Impervious Greatwurm            [G]   16.0       16.0
3         Emrakul and Chatterfang         [B, G]   15.0       15.0
4                 Worldspine Wurm            [G]   15.0       15.0
5         Emrakul, the Aeons Torn             []   15.0       15.0
6                   The Last Ride            [B]   13.0       13.0
7                  Death's Shadow            [B]   13.0       13.0
8             Krosan Cloudscraper            [G]   13.0       13.0
9       Emrakul, the Promised End             []   13.0       13.0
10  Kozilek, the Great Distortion             []   12.0       12.0
11        Ghalta, Stampede Tyrant            [G]   12.0       12.0
12             Yargle and Multani         [B, G]   18.0        6.0
13      Kozilek, Butcher of Truth             []   12.0       

In [ ]:
#Rarity types
my_query =  '''
    SELECT 
        DISTINCT rarity
    FROM df
    '''
results = db.query(my_query).to_df()
print(results)


     rarity
0      rare
1   special
2     bonus
3    common
4  uncommon
5    mythic


In [ ]:
#Third Question
rarities = ['rare', 'special', 'bonus', 'common', 'uncommon', 'mythic']
for i in range(6):
    print(rarities[i].upper())
    my_query = f"SELECT set_name, COUNT(rarity) AS rarity_count FROM df WHERE rarity = '{rarities[i]}' GROUP BY set_name ORDER BY rarity_count DESC LIMIT(3);"
    results = db.query(my_query).to_df()
    print(f"{results}\n")


RARE
                                      set_name  rarity_count
0                                UNKNOWN EVENT           229
1                            COMMANDER MASTERS           182
2  COMMANDER LEGENDS: BATTLE FOR BALDUR'S GATE           181

SPECIAL
                                      set_name  rarity_count
0                      TIME SPIRAL TIMESHIFTED            60
1                            COMMANDER MASTERS             1
2  COMMANDER LEGENDS: BATTLE FOR BALDUR'S GATE             1

BONUS
          set_name  rarity_count
0  VINTAGE MASTERS             9

COMMON
                                      set_name  rarity_count
0                                  FOUNDATIONS           141
1  COMMANDER LEGENDS: BATTLE FOR BALDUR'S GATE           136
2                                    JUMPSTART           130

UNCOMMON
                                      set_name  rarity_count
0                                  FOUNDATIONS           172
1                            COMMANDER MA

In [ ]:
my_query =  '''
    SELECT 
        set_name,
        COUNT(oracle_id) AS amount
    FROM df
    GROUP BY set_name
    ORDER BY amount DESC;
    '''
results = db.query(my_query).to_df()
print(results)

                                        set_name  amount
0                              COMMANDER MASTERS     514
1    COMMANDER LEGENDS: BATTLE FOR BALDUR'S GATE     488
2                                    FOUNDATIONS     467
3                                  UNKNOWN EVENT     437
4                              MODERN HORIZONS 3     305
..                                           ...     ...
504                            CONSPIRACY TOKENS       1
505                            MICROPROSE PROMOS       1
506                            BATTLEBOND TOKENS       1
507                               MAGICFEST 2025       1
508                 OATH OF THE GATEWATCH TOKENS       1

[509 rows x 2 columns]


In [ ]:
#Fourth Question
#It doesnt count one more into flying if a card has another keywords
my_query = """
    SELECT
        keywords,
        COUNT(keywords) AS amount
    FROM df
    GROUP BY keywords
    ORDER BY amount DESC;

"""
results = db.query(my_query).to_df()
print(results)

                                keywords  amount
0                                     []   19021
1                               [Flying]    1626
2                              [Enchant]     986
3                                [Equip]     418
4                              [Trample]     403
...                                  ...     ...
2481  [Lifelink, Underdog, First strike]       1
2482         [Protection, Double strike]       1
2483               [Role token, Trample]       1
2484  [Protection, Trample, Monstrosity]       1
2485                    [Trample, Equip]       1

[2486 rows x 2 columns]


In [ ]:
#Solution
df_exploded = df.explode("keywords")  # Split list into rows
keyword_counts = df_exploded["keywords"].value_counts().head(30)
print(keyword_counts)


keywords
Flying          3099
Enchant         1232
Trample          940
Haste            653
Vigilance        640
Equip            532
Mill             528
Flash            525
Scry             433
First strike     377
Cycling          362
Transform        357
Treasure         348
Reach            345
Menace           341
Lifelink         335
Defender         311
Deathtouch       306
Kicker           227
Protection       214
Flashback        178
Ward             171
Crew             162
Morph            153
Food             150
Conjure          150
Surveil          148
Landfall         145
Investigate      142
Landwalk         136
Name: count, dtype: int64


In [414]:
#fifth Question
df['keyword_counting'] = df['keywords'].apply(len)
some_columns = df[['name', 'keyword_counting','legalities']].sort_values(by='keyword_counting', ascending=False).head(10)
print(some_columns)


                                                    name  keyword_counting  \
21304                                          Old Fogey                10   
13990                                Odric, Blood-Cursed                10   
21515                               Sire of Seven Deaths                 7   
27103                                    Lunar Hatchling                 7   
33486  Slicer, Hired Muscle // Slicer, High-Speed Ant...                 7   
22361     Throne of the Grim Captain // The Grim Captain                 7   
12965                                 Injector Crocodile                 6   
14031                                      Chromanticore                 6   
31916                                    Igneous Pouncer                 6   
8032                             Emrakul and Chatterfang                 6   

                                              legalities  
21304  {'standard': 'not_legal', 'future': 'not_legal...  
13990  {'standard': 'no

In [421]:
#Sixth Question
df['name_len'] = df['name'].apply(len)
bigger = df[['name', 'name_len',]].sort_values(by='name_len', ascending=False).head(1)
smaller = df[['name', 'name_len',]].sort_values(by='name_len', ascending=True).head(1)
print(f"smaller: {smaller}\nbigger: {bigger}")


smaller:       name  name_len
32009    X         1
bigger:                                                     name  name_len
19852  Our Market Research Shows That Players Like Re...       141


In [553]:
#Seventh Question
df['oracle_txt_len'] = df['oracle_text'].astype(str).apply(len)
results = df[['name','oracle_txt_len']].sort_values(by='oracle_txt_len', ascending=False).head(15)
print(results)


                                     name  oracle_txt_len
940              Baldur's Gate Wilderness            1487
4608                          Bureaucracy             774
7828                        Booster Blitz             763
6460              Dungeon of the Mad Mage             707
27221  Shadows Over Innistrad Checklist 1             706
11617             Eldritch Moon Checklist             676
13222                    Brian Selden Bio             662
1389                         Rules Lawyer             645
15854                   Greater Morphling             639
28232                     Peer Kröger Bio             632
31871                  Dave Humpherys Bio             630
27489                  Master of the Hunt             627
21961                 Innistrad Checklist             610
2947                    Dance of the Dead             609
22025                     Jan Tomcani Bio             598


## 2. Answers:
### 1. If we consider colorless as a color, it wins with 5,589 cards. Otherwise, White and Black are the top single colors, each with 4,710 cards. For two-color combinations, Dimir (Black and Blue) comes in first with 480 cards. For rainbow (all five colors), there are 140 cards. For three-color combinations, Naya (Green, Red, and White) is the most represented with 116 cards. Finally, four-color combinations aren't as common, but Green, Red, Blue and White is the top four-color combination with only 8 cards.
### 2. As long as Big Furry(99/99, Black) is a two-cards creature that isn't legal in any format and Marit Lage(20/20, Black) is a token, the strogest creature card considering power + tougness is Impervious Greatwurm(16/16, Green). It's interisting to know there is a creature that deals more damage than Impervious Greatwurm, Yargle and Multani(18/6, Golgari(Black and Green)). https://gatherer.wizards.com/pages/card/Details.aspx?multiverseid=455613
### 3. Commander Masters is the set with more rare and mythic cards.
### 4. The most commonly used keyword is Flying.
### 5. Considering keyword written as the own card abilities, the card with ther most keywords is Sire of Seven Deaths. https://gatherer.wizards.com/pages/Card/Details.aspx?multiverseid=679078
### 6. Respectvely, "Our Market Research Shows That Players Like Really Long Card Names So We Made this Card to Have the Absolute Longest Card Name Ever Elemental" and "X".
### 7. The card with the longest oracle text is Dance of the Dead. https://gatherer.wizards.com/pages/card/Details.aspx?multiverseid=184612